In [17]:
import os
import sys
import copy
import pickle
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras.api._v2.keras as keras
import keras.backend as K
from keras.optimizers.legacy import Adam
from keras.optimizers import SGD
from keras import callbacks
from keras.models import Model
from keras.layers import Layer, Dense, Input, Activation, Lambda, BatchNormalization, Conv1D, SpatialDropout1D, add, GlobalAveragePooling1D, LSTM, Dense, concatenate, TimeDistributed
from keras.activations import sigmoid
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score
import tensorflow as tf
from tensorflow.keras import optimizers

In [12]:
# Load multimodal data

from collections import Counter
# emotion_counter = Counter()

def createOneHot(train_label, test_label):
    
    maxlen = int(max(train_label.max(), test_label.max()))

    train = np.zeros((train_label.shape[0], train_label.shape[1], maxlen + 1))
    test = np.zeros((test_label.shape[0], test_label.shape[1], maxlen + 1))

    for i in range(train_label.shape[0]):
        for j in range(train_label.shape[1]):
            train[i, j, train_label[i, j]] = 1

    for i in range(test_label.shape[0]):
        for j in range(test_label.shape[1]):
            test[i, j, test_label[i, j]] = 1

    return train, test

# Load data
f = open("dataset/iemocap/raw/IEMOCAP_features_raw.pkl", "rb")
u = pickle._Unpickler(f)
u.encoding = 'latin1'
videoIDs, videoSpeakers, videoLabels, videoText, videoAudio, videoVisual, videoSentence, trainVid, testVid = u.load()
# label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}

print('Number of training samples: ',len(trainVid))
print('Number of testing samples: ',len(testVid))

train_audio = []
train_text = []
train_visual = []
train_seq_len = []
train_label = []

test_audio = []
test_text = []
test_visual = []
test_seq_len = []
test_label = []
for vid in trainVid:
    train_seq_len.append(len(videoIDs[vid]))
for vid in testVid:
    test_seq_len.append(len(videoIDs[vid]))

max_len = max(max(train_seq_len), max(test_seq_len))
print('Max length of sequences: ', max_len)
print()

for vid in trainVid:
    # emotion_counter.update(videoLabels[vid])
    train_label.append(videoLabels[vid] + [0] * (max_len - len(videoIDs[vid])))
    pad = [np.zeros(videoText[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    text = np.stack(videoText[vid] + pad, axis=0)
    train_text.append(text)

    pad = [np.zeros(videoAudio[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    audio = np.stack(videoAudio[vid] + pad, axis=0)
    train_audio.append(audio)

    pad = [np.zeros(videoVisual[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    video = np.stack(videoVisual[vid] + pad, axis=0)
    train_visual.append(video)

for vid in testVid:
    test_label.append(videoLabels[vid] + [0] * (max_len - len(videoIDs[vid])))
    pad = [np.zeros(videoText[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    text = np.stack(videoText[vid] + pad, axis=0)
    test_text.append(text)

    pad = [np.zeros(videoAudio[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    audio = np.stack(videoAudio[vid] + pad, axis=0)
    test_audio.append(audio)

    pad = [np.zeros(videoVisual[vid][0].shape)] * (max_len - len(videoIDs[vid]))
    video = np.stack(videoVisual[vid] + pad, axis=0)
    test_visual.append(video)

train_text = np.stack(train_text, axis=0)
train_audio = np.stack(train_audio, axis=0)
train_visual = np.stack(train_visual, axis=0)
print(f"Train text shape: {train_text.shape[0]} samples, {train_text.shape[1]} timesteps, {train_text.shape[2]} features")
print(f"Train audio shape: {train_audio.shape[0]} samples, {train_audio.shape[1]} timesteps, {train_audio.shape[2]} features")
print(f"Train visual shape: {train_visual.shape[0]} samples, {train_visual.shape[1]} timesteps, {train_visual.shape[2]} features")
print()

test_text = np.stack(test_text, axis=0)
test_audio = np.stack(test_audio, axis=0)
test_visual = np.stack(test_visual, axis=0)
print(f"Test text shape: {test_text.shape[0]} samples, {test_text.shape[1]} timesteps, {test_text.shape[2]} features")
print(f"Test audio shape: {test_audio.shape[0]} samples, {test_audio.shape[1]} timesteps, {test_audio.shape[2]} features")
print(f"Test visual shape: {test_visual.shape[0]} samples, {test_visual.shape[1]} timesteps, {test_visual.shape[2]} features")
print()

train_label = np.array(train_label)
test_label = np.array(test_label)
train_seq_len = np.array(train_seq_len)
test_seq_len = np.array(test_seq_len)

train_mask = np.zeros((train_text.shape[0], train_text.shape[1]), dtype='float')
for i in range(len(train_seq_len)):
    train_mask[i, :train_seq_len[i]] = 1.0

test_mask = np.zeros((test_text.shape[0], test_text.shape[1]), dtype='float')
for i in range(len(test_seq_len)):
    test_mask[i, :test_seq_len[i]] = 1.0

train_label, test_label = createOneHot(train_label, test_label)
train_data = np.concatenate((train_audio, train_visual, train_text), axis=-1)
test_data = np.concatenate((test_audio, test_visual, test_text), axis=-1)
audio_text_train_data = np.concatenate((train_audio, train_text), axis=-1)
audio_text_test_data = np.concatenate((test_audio, test_text), axis=-1)

print(f"Train data shape: {train_data.shape[0]} samples, {train_data.shape[1]} timesteps, {train_data.shape[2]} features")
print(f"Train label shape: {train_label.shape[0]} samples, {train_label.shape[1]} timesteps, {train_label.shape[2]} classes")
print()
print(f"Test data shape: {test_data.shape[0]} samples, {test_data.shape[1]} timesteps, {test_data.shape[2]} features")
print(f"Test label shape: {test_label.shape[0]} samples, {test_label.shape[1]} timesteps, {test_label.shape[2]} classes")
print()

# for emotion, count in emotion_counter.items():
#   print(f"{emotion}: {count}")

Number of training samples:  120
Number of testing samples:  31
Max length of sequences:  110

Train text shape: 120 samples, 110 timesteps, 100 features
Train audio shape: 120 samples, 110 timesteps, 100 features
Train visual shape: 120 samples, 110 timesteps, 512 features

Test text shape: 31 samples, 110 timesteps, 100 features
Test audio shape: 31 samples, 110 timesteps, 100 features
Test visual shape: 31 samples, 110 timesteps, 512 features

Train data shape: 120 samples, 110 timesteps, 712 features
Train label shape: 120 samples, 110 timesteps, 6 classes

Test data shape: 31 samples, 110 timesteps, 712 features
Test label shape: 31 samples, 110 timesteps, 6 classes



In [66]:
class BaseModel:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        pass

    def compile(self, lr=0.001):
        self.model.compile(
            optimizer=Adam(lr=lr),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train(self, x_train, y_train, x_val, y_val, epochs=10, batch_size=32):
        return self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=epochs,
            batch_size=batch_size
        )

    def evaluate(self, x_test, y_test):
        predictions = self.model.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        # Return a dictionary of metrics
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
        }


In [67]:
class EarlyFusionModel(BaseModel):
    def build_model(self):
        inputs = Input(shape=self.input_shape)
        x = LSTM(64, return_sequences=True)(inputs)
        x = LSTM(64, return_sequences=True)(x)
        outputs = Dense(self.num_classes, activation='softmax')(x)
        return Model(inputs, outputs)
    
    def train_and_evaluate_on_test(self, x_train, y_train, x_test, y_test, epochs=10, batch_size=32):
        # Train the model on the entire training dataset
        history = self.model.fit(
            x_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )

        # Evaluate the model on the test dataset
        test_metrics = self.evaluate(x_test, y_test)

        return history, test_metrics

In [68]:
# Perform cross validation
def cross_val_training(model_class, data, labels, num_folds=5, epochs=10, batch_size=32):
    kf = KFold(n_splits=num_folds, shuffle=True)
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for train_index, val_index in kf.split(data):
        x_train_fold, x_val_fold = data[train_index], data[val_index]
        y_train_fold, y_val_fold = labels[train_index], labels[val_index]

        model = model_class(input_shape=x_train_fold.shape[1:], num_classes=y_train_fold.shape[2])
        model.compile(lr=0.001)
        model.train(x_train_fold, y_train_fold, x_val_fold, y_val_fold, epochs=epochs, batch_size=batch_size)
        
        # Evaluate the model on the validation fold
        y_pred = model.model.predict(x_val_fold)
        y_pred = np.argmax(y_pred, axis=-1).flatten()
        y_true = np.argmax(y_val_fold, axis=-1).flatten()
        
        # Calculate and store metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
        
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
    # Calculate average metrics across all folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    # Calculate standard deviations
    std_accuracy = np.std(accuracies)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)
    std_f1 = np.std(f1_scores)

    print(f"K-Fold Cross-Validation Results ({num_folds} folds):")
    print(f"Average Accuracy: {avg_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f} ± {std_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f} ± {std_recall:.4f}")
    print(f"Average F1-Score: {avg_f1:.4f} ± {std_f1:.4f}")

cross_val_training(EarlyFusionModel, train_data, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:12:17.683280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:17.907597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:18.065967: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:18.285376: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:18.682109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7168 - accuracy: 0.5271

2024-03-16 15:12:19.545974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:19.626058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:19.745773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 3s 490ms/step - loss: 1.7168 - accuracy: 0.5271 - val_loss: 1.6029 - val_accuracy: 0.7515
Epoch 2/10
3/3 [==============================] - 0s 168ms/step - loss: 1.5259 - accuracy: 0.7978 - val_loss: 1.4422 - val_accuracy: 0.7902
Epoch 3/10
3/3 [==============================] - 0s 141ms/step - loss: 1.3360 - accuracy: 0.8185 - val_loss: 1.2470 - val_accuracy: 0.7913
Epoch 4/10
3/3 [==============================] - 0s 87ms/step - loss: 1.0804 - accuracy: 0.8246 - val_loss: 0.9111 - val_accuracy: 0.7803
Epoch 5/10
3/3 [==============================] - 0s 135ms/step - loss: 0.7025 - accuracy: 0.8277 - val_loss: 0.6170 - val_accuracy: 0.7814
Epoch 6/10
3/3 [==============================] - 0s 64ms/step - loss: 0.5089 - accuracy: 0.8342 - val_loss: 0.5302 - val_accuracy: 0.8242
Epoch 7/10
3/3 [==============================] - 0s 91ms/step - loss: 0.4504 - accuracy: 0.8483 - val_loss: 0.4937 - val_accuracy: 0.8299
Epoch 8/10
3/3 [==================

...

3/3 [==============================] - ETA: 0s - loss: 1.7460 - accuracy: 0.4337

2024-03-16 15:12:46.466616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:46.552783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:46.727598: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 4s 523ms/step - loss: 1.7460 - accuracy: 0.4337 - val_loss: 1.6170 - val_accuracy: 0.7205
Epoch 2/10
3/3 [==============================] - 0s 171ms/step - loss: 1.5556 - accuracy: 0.7812 - val_loss: 1.4536 - val_accuracy: 0.7841
Epoch 3/10
3/3 [==============================] - 0s 179ms/step - loss: 1.3952 - accuracy: 0.8105 - val_loss: 1.2831 - val_accuracy: 0.8125
Epoch 4/10
3/3 [==============================] - 0s 106ms/step - loss: 1.1938 - accuracy: 0.8283 - val_loss: 1.0185 - val_accuracy: 0.8299
Epoch 5/10
3/3 [==============================] - 0s 68ms/step - loss: 0.8589 - accuracy: 0.8440 - val_loss: 0.6469 - val_accuracy: 0.8364
Epoch 6/10
3/3 [==============================] - 0s 111ms/step - loss: 0.5364 - accuracy: 0.8504 - val_loss: 0.5138 - val_accuracy: 0.8417
Epoch 7/10
3/3 [==============================] - 0s 78ms/step - loss: 0.4512 - accuracy: 0.8580 - val_loss: 0.4703 - val_accuracy: 0.8477
Epoch 8/10
3/3 [=================

2024-03-16 15:12:49.958378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:50.023741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:50.224018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 662ms/step
K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.8725 ± 0.0116
Average Precision: 0.7678 ± 0.0207
Average Recall: 0.7811 ± 0.0174
Average F1-Score: 0.7720 ± 0.0181


In [69]:
# Train and evaluate based on train and test data
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_data.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_data, train_label, test_data, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:12:53.883385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:54.077559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:54.308008: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:54.732859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:55.178551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 3s 279ms/step - loss: 1.7512 - accuracy: 0.4660
Epoch 2/10
4/4 [==============================] - 1s 129ms/step - loss: 1.5435 - accuracy: 0.7802
Epoch 3/10
4/4 [==============================] - 0s 73ms/step - loss: 1.3521 - accuracy: 0.8039
Epoch 4/10
4/4 [==============================] - 0s 74ms/step - loss: 1.0340 - accuracy: 0.8298
Epoch 5/10
4/4 [==============================] - 0s 77ms/step - loss: 0.6247 - accuracy: 0.8414
Epoch 6/10
4/4 [==============================] - 0s 84ms/step - loss: 0.4831 - accuracy: 0.8524
Epoch 7/10
4/4 [==============================] - 0s 46ms/step - loss: 0.4219 - accuracy: 0.8608
Epoch 8/10
4/4 [==============================] - 0s 49ms/step - loss: 0.3865 - accuracy: 0.8746
Epoch 9/10
4/4 [==============================] - 0s 67ms/step - loss: 0.3544 - accuracy: 0.8867
Epoch 10/10
4/4 [==============================] - 0s 68ms/step - loss: 0.3228 - accuracy: 0.8969


2024-03-16 15:12:59.262132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:59.328255: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:12:59.529972: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 688ms/step
Test Metrics:
Accuracy: 0.7657
Precision: 0.6927
Recall: 0.5655
F1 Score: 0.5995


In [70]:
class LateFusionModel(BaseModel):
    def __init__(self, input_shapes, num_classes):
        self.input_shapes = input_shapes  # This will be a list of input shapes for each modality
        super().__init__(input_shape=None, num_classes=num_classes)  # We pass None to input_shape

    def build_model(self):
        inputs = [Input(shape=shape) for shape in self.input_shapes]
        features = [LSTM(32, return_sequences=True)(input_tensor) for input_tensor in inputs]
        combined_features = concatenate(features, axis=-1)
        outputs = TimeDistributed(Dense(self.num_classes, activation='softmax'))(combined_features)
        model = Model(inputs=inputs, outputs=outputs)
        return model
    
    def train_and_evaluate_on_test(self):
        history = self.model.fit(
            [train_audio, train_visual, train_text], train_label,
            epochs=10,
            batch_size=32,
            verbose=1
        )
        test_metrics = self.evaluate([test_audio, test_visual, test_text], test_label)
        return history, test_metrics

In [71]:
def cross_val_training_late_fusion(model_class, modalities_data, labels, num_folds=5, epochs=10, batch_size=32):
    kf = KFold(n_splits=num_folds, shuffle=True)
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Ensure modalities_data is a list of datasets, one for each modality
    if not isinstance(modalities_data, list):
        raise ValueError("modalities_data should be a list of datasets, one for each modality.")

    for train_index, val_index in kf.split(modalities_data[0]):  # Use the first modality to generate indices
        # Prepare training and validation data for each modality
        x_train_fold = [modality[train_index] for modality in modalities_data]
        x_val_fold = [modality[val_index] for modality in modalities_data]
        y_train_fold, y_val_fold = labels[train_index], labels[val_index]

        # Instantiate the model for each fold. Pass None to input_shape for LateFusionModel
        model = model_class(input_shapes=[modality.shape[1:] for modality in x_train_fold], num_classes=y_train_fold.shape[-1])
        model.compile(lr=0.001)

        # Train the model using the custom train method, which internally uses .fit()
        model.train(x_train_fold, y_train_fold, x_val_fold, y_val_fold, epochs=epochs, batch_size=batch_size)

        # Evaluate the model on the validation fold using the custom evaluate method
        metrics = model.evaluate(x_val_fold, y_val_fold)

        # Store the evaluation metrics
        accuracies.append(metrics['accuracy'])
        precisions.append(metrics['precision'])
        recalls.append(metrics['recall'])
        f1_scores.append(metrics['f1_score'])
    
    # Calculate and print the average and standard deviation of the metrics across all folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    # Calculate standard deviations
    std_accuracy = np.std(accuracies)
    std_precision = np.std(precisions)
    std_recall = np.std(recalls)
    std_f1 = np.std(f1_scores)

    print(f"K-Fold Cross-Validation Results ({num_folds} folds):")
    print(f"Average Accuracy: {avg_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f} ± {std_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f} ± {std_recall:.4f}")
    print(f"Average F1-Score: {avg_f1:.4f} ± {std_f1:.4f}")

cross_val_training_late_fusion(LateFusionModel, [train_audio, train_visual, train_text], train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:16:00.414519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:00.696117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:00.698337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:00.698827: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:01.619847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-0

3/3 [==============================] - ETA: 0s - loss: 1.7611 - accuracy: 0.4263

2024-03-16 15:16:03.560582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:03.661326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:03.661994: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:03.662562: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 5s 822ms/step - loss: 1.7611 - accuracy: 0.4263 - val_loss: 1.7014 - val_accuracy: 0.6367
Epoch 2/10
3/3 [==============================] - 1s 312ms/step - loss: 1.6684 - accuracy: 0.6903 - val_loss: 1.6154 - val_accuracy: 0.7568
Epoch 3/10
3/3 [==============================] - 0s 166ms/step - loss: 1.5823 - accuracy: 0.7956 - val_loss: 1.5311 - val_accuracy: 0.8280
Epoch 4/10
3/3 [==============================] - 0s 133ms/step - loss: 1.4947 - accuracy: 0.8591 - val_loss: 1.4387 - val_accuracy: 0.8689
Epoch 5/10
3/3 [==============================] - 0s 137ms/step - loss: 1.3983 - accuracy: 0.8938 - val_loss: 1.3310 - val_accuracy: 0.8939
Epoch 6/10
3/3 [==============================] - 0s 146ms/step - loss: 1.2795 - accuracy: 0.9068 - val_loss: 1.1918 - val_accuracy: 0.8898
Epoch 7/10
3/3 [==============================] - 0s 89ms/step - loss: 1.1154 - accuracy: 0.8860 - val_loss: 0.9933 - val_accuracy: 0.8439
Epoch 8/10
3/3 [================

...

3/3 [==============================] - ETA: 0s - loss: 1.8323 - accuracy: 0.3865

2024-03-16 15:16:40.599372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:40.700869: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:40.701348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:40.702622: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 5s 872ms/step - loss: 1.8323 - accuracy: 0.3865 - val_loss: 1.7405 - val_accuracy: 0.6360
Epoch 2/10
3/3 [==============================] - 1s 438ms/step - loss: 1.7181 - accuracy: 0.6082 - val_loss: 1.6525 - val_accuracy: 0.7405
Epoch 3/10
3/3 [==============================] - 1s 175ms/step - loss: 1.6127 - accuracy: 0.7186 - val_loss: 1.5662 - val_accuracy: 0.7989
Epoch 4/10
3/3 [==============================] - 0s 114ms/step - loss: 1.5119 - accuracy: 0.7915 - val_loss: 1.4765 - val_accuracy: 0.8481
Epoch 5/10
3/3 [==============================] - 0s 115ms/step - loss: 1.4115 - accuracy: 0.8390 - val_loss: 1.3739 - val_accuracy: 0.8818
Epoch 6/10
3/3 [==============================] - 0s 117ms/step - loss: 1.3003 - accuracy: 0.8633 - val_loss: 1.2462 - val_accuracy: 0.8875
Epoch 7/10
3/3 [==============================] - 0s 96ms/step - loss: 1.1642 - accuracy: 0.8648 - val_loss: 1.0682 - val_accuracy: 0.8739
Epoch 8/10
3/3 [================

2024-03-16 15:16:45.204298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:45.290444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:45.290471: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:16:45.290498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 766ms/step
K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.8648 ± 0.0256
Average Precision: 0.8881 ± 0.0183
Average Recall: 0.7141 ± 0.0432
Average F1-Score: 0.7732 ± 0.0311


In [72]:
# Example: Instantiating and compiling the LateFusionModel
input_shapes = [train_audio.shape[1:], train_visual.shape[1:], train_text.shape[1:]]  # Adjust as per your data
num_classes = train_label.shape[-1]  # Assuming one-hot encoded labels

model = LateFusionModel(input_shapes=input_shapes, num_classes=num_classes)
model.compile(lr=0.001)  # Make sure your BaseModel's compile method properly configures the underlying Keras model

history, test_metrics = model.train_and_evaluate_on_test()

print("Test Metrics for Late Fusion Model:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:18:25.214959: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:25.536325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:25.536364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:25.536383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:26.516749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-0

4/4 [==============================] - 5s 517ms/step - loss: 1.7841 - accuracy: 0.4283
Epoch 2/10
4/4 [==============================] - 1s 198ms/step - loss: 1.6521 - accuracy: 0.7025
Epoch 3/10
4/4 [==============================] - 1s 193ms/step - loss: 1.5379 - accuracy: 0.8000
Epoch 4/10
4/4 [==============================] - 0s 140ms/step - loss: 1.4183 - accuracy: 0.8458
Epoch 5/10
4/4 [==============================] - 0s 91ms/step - loss: 1.2744 - accuracy: 0.8739
Epoch 6/10
4/4 [==============================] - 0s 85ms/step - loss: 1.0724 - accuracy: 0.8741
Epoch 7/10
4/4 [==============================] - 0s 60ms/step - loss: 0.7988 - accuracy: 0.8513
Epoch 8/10
4/4 [==============================] - 0s 48ms/step - loss: 0.5861 - accuracy: 0.8394
Epoch 9/10
4/4 [==============================] - 0s 40ms/step - loss: 0.4720 - accuracy: 0.8792
Epoch 10/10
4/4 [==============================] - 0s 50ms/step - loss: 0.4081 - accuracy: 0.9189


2024-03-16 15:18:32.914517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:33.002422: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:33.002992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:18:33.003037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 754ms/step
Test Metrics for Late Fusion Model:
Accuracy: 0.7554
Precision: 0.6290
Recall: 0.5856
F1 Score: 0.5891


In [73]:
# Try with just audio data
cross_val_training(EarlyFusionModel, train_audio, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:19:34.843413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:19:35.020940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:19:35.304012: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:19:35.665435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:19:36.098029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7782 - accuracy: 0.4376

2024-03-16 15:19:37.491131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:19:37.580506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:19:37.795985: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 4s 692ms/step - loss: 1.7782 - accuracy: 0.4376 - val_loss: 1.7387 - val_accuracy: 0.6258
Epoch 2/10
3/3 [==============================] - 0s 132ms/step - loss: 1.7183 - accuracy: 0.6062 - val_loss: 1.6620 - val_accuracy: 0.6186
Epoch 3/10
3/3 [==============================] - 1s 186ms/step - loss: 1.6161 - accuracy: 0.6026 - val_loss: 1.5078 - val_accuracy: 0.6152
Epoch 4/10
3/3 [==============================] - 0s 161ms/step - loss: 1.4195 - accuracy: 0.6012 - val_loss: 1.2102 - val_accuracy: 0.6159
Epoch 5/10
3/3 [==============================] - 0s 88ms/step - loss: 1.1685 - accuracy: 0.6037 - val_loss: 1.0242 - val_accuracy: 0.6295
Epoch 6/10
3/3 [==============================] - 0s 76ms/step - loss: 1.0419 - accuracy: 0.6218 - val_loss: 0.9292 - val_accuracy: 0.6693
Epoch 7/10
3/3 [==============================] - 0s 106ms/step - loss: 0.9665 - accuracy: 0.6454 - val_loss: 0.8826 - val_accuracy: 0.6640
Epoch 8/10
3/3 [=================

...

2024-03-16 15:20:07.572999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:07.654026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:07.893489: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 4s 709ms/step - loss: 1.7789 - accuracy: 0.4705 - val_loss: 1.7401 - val_accuracy: 0.6015
Epoch 2/10
3/3 [==============================] - 1s 189ms/step - loss: 1.7200 - accuracy: 0.6038 - val_loss: 1.6629 - val_accuracy: 0.5894
Epoch 3/10
3/3 [==============================] - 0s 177ms/step - loss: 1.6320 - accuracy: 0.6007 - val_loss: 1.5240 - val_accuracy: 0.5902
Epoch 4/10
3/3 [==============================] - 0s 102ms/step - loss: 1.4613 - accuracy: 0.6006 - val_loss: 1.2862 - val_accuracy: 0.5924
Epoch 5/10
3/3 [==============================] - 0s 124ms/step - loss: 1.2182 - accuracy: 0.6021 - val_loss: 1.1300 - val_accuracy: 0.5966
Epoch 6/10
3/3 [==============================] - 0s 161ms/step - loss: 1.0843 - accuracy: 0.6087 - val_loss: 1.0336 - val_accuracy: 0.6246
Epoch 7/10
3/3 [==============================] - 0s 109ms/step - loss: 0.9836 - accuracy: 0.6287 - val_loss: 0.9601 - val_accuracy: 0.6432
Epoch 8/10
3/3 [===============

2024-03-16 15:20:11.524917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:11.589774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:11.803123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 700ms/step
K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.6742 ± 0.0339
Average Precision: 0.3638 ± 0.0416
Average Recall: 0.4376 ± 0.0208
Average F1-Score: 0.3688 ± 0.0233


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [75]:
cross_val_training(EarlyFusionModel, train_visual, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:20:48.741976: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:48.925939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:49.178592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:49.535699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:49.965963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7761 - accuracy: 0.4501

2024-03-16 15:20:51.009208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:51.092589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:20:51.317016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 4s 604ms/step - loss: 1.7761 - accuracy: 0.4501 - val_loss: 1.7574 - val_accuracy: 0.5830
Epoch 2/10
3/3 [==============================] - 1s 225ms/step - loss: 1.7156 - accuracy: 0.6019 - val_loss: 1.6829 - val_accuracy: 0.5818
Epoch 3/10
3/3 [==============================] - 0s 127ms/step - loss: 1.5695 - accuracy: 0.6021 - val_loss: 1.5212 - val_accuracy: 0.5818
Epoch 4/10
3/3 [==============================] - 0s 168ms/step - loss: 1.3227 - accuracy: 0.6021 - val_loss: 1.3722 - val_accuracy: 0.5818
Epoch 5/10
3/3 [==============================] - 0s 141ms/step - loss: 1.1980 - accuracy: 0.6021 - val_loss: 1.3288 - val_accuracy: 0.5818
Epoch 6/10
3/3 [==============================] - 0s 107ms/step - loss: 1.1665 - accuracy: 0.6022 - val_loss: 1.2508 - val_accuracy: 0.5822
Epoch 7/10
3/3 [==============================] - 0s 85ms/step - loss: 1.0945 - accuracy: 0.6023 - val_loss: 1.1268 - val_accuracy: 0.5864
Epoch 8/10
3/3 [================

...

2024-03-16 15:21:20.820064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:21:20.908680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:21:21.120650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 3s 604ms/step - loss: 1.7799 - accuracy: 0.4525 - val_loss: 1.7470 - val_accuracy: 0.6295
Epoch 2/10
3/3 [==============================] - 1s 258ms/step - loss: 1.7307 - accuracy: 0.5906 - val_loss: 1.6474 - val_accuracy: 0.6284
Epoch 3/10
3/3 [==============================] - 0s 172ms/step - loss: 1.5964 - accuracy: 0.5904 - val_loss: 1.3668 - val_accuracy: 0.6284
Epoch 4/10
3/3 [==============================] - 0s 129ms/step - loss: 1.3390 - accuracy: 0.5904 - val_loss: 1.1118 - val_accuracy: 0.6284
Epoch 5/10
3/3 [==============================] - 0s 116ms/step - loss: 1.1965 - accuracy: 0.5905 - val_loss: 0.9924 - val_accuracy: 0.6428
Epoch 6/10
3/3 [==============================] - 0s 99ms/step - loss: 1.0900 - accuracy: 0.6190 - val_loss: 0.9825 - val_accuracy: 0.6716
Epoch 7/10
3/3 [==============================] - 0s 71ms/step - loss: 1.0443 - accuracy: 0.6339 - val_loss: 0.8832 - val_accuracy: 0.6739
Epoch 8/10
3/3 [=================

2024-03-16 15:21:24.680187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:21:24.743912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:21:24.958729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 696ms/step
K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.6480 ± 0.0251
Average Precision: 0.1957 ± 0.0273
Average Recall: 0.2934 ± 0.0172
Average F1-Score: 0.2185 ± 0.0198


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
cross_val_training(EarlyFusionModel, train_text, train_label, num_folds=5, epochs=10, batch_size=32)

Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:22:44.197177: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:22:44.362592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:22:44.668789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:22:45.035232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:22:45.465208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - ETA: 0s - loss: 1.7438 - accuracy: 0.4625

2024-03-16 15:22:46.530273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:22:46.621736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:22:46.843969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 4s 626ms/step - loss: 1.7438 - accuracy: 0.4625 - val_loss: 1.6604 - val_accuracy: 0.7034
Epoch 2/10
3/3 [==============================] - 1s 215ms/step - loss: 1.6060 - accuracy: 0.7540 - val_loss: 1.5406 - val_accuracy: 0.7837
Epoch 3/10
3/3 [==============================] - 1s 200ms/step - loss: 1.4790 - accuracy: 0.7912 - val_loss: 1.4069 - val_accuracy: 0.8019
Epoch 4/10
3/3 [==============================] - 0s 141ms/step - loss: 1.3309 - accuracy: 0.8095 - val_loss: 1.2274 - val_accuracy: 0.8170
Epoch 5/10
3/3 [==============================] - 0s 109ms/step - loss: 1.1047 - accuracy: 0.8226 - val_loss: 0.9307 - val_accuracy: 0.8106
Epoch 6/10
3/3 [==============================] - 0s 97ms/step - loss: 0.7407 - accuracy: 0.8317 - val_loss: 0.6309 - val_accuracy: 0.7917
Epoch 7/10
3/3 [==============================] - 0s 105ms/step - loss: 0.5095 - accuracy: 0.8366 - val_loss: 0.5697 - val_accuracy: 0.8106
Epoch 8/10
3/3 [================

...

3/3 [==============================] - ETA: 0s - loss: 1.7508 - accuracy: 0.4422

2024-03-16 15:23:16.992916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:23:17.069941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:23:17.282402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 4s 666ms/step - loss: 1.7508 - accuracy: 0.4422 - val_loss: 1.6439 - val_accuracy: 0.7352
Epoch 2/10
3/3 [==============================] - 0s 137ms/step - loss: 1.5977 - accuracy: 0.7555 - val_loss: 1.5083 - val_accuracy: 0.8015
Epoch 3/10
3/3 [==============================] - 0s 151ms/step - loss: 1.4500 - accuracy: 0.8024 - val_loss: 1.3408 - val_accuracy: 0.8129
Epoch 4/10
3/3 [==============================] - 0s 141ms/step - loss: 1.2508 - accuracy: 0.8153 - val_loss: 1.0640 - val_accuracy: 0.8098
Epoch 5/10
3/3 [==============================] - 0s 68ms/step - loss: 0.9147 - accuracy: 0.8208 - val_loss: 0.7055 - val_accuracy: 0.8178
Epoch 6/10
3/3 [==============================] - 0s 149ms/step - loss: 0.6191 - accuracy: 0.8227 - val_loss: 0.5838 - val_accuracy: 0.8178
Epoch 7/10
3/3 [==============================] - 0s 74ms/step - loss: 0.5257 - accuracy: 0.8340 - val_loss: 0.5245 - val_accuracy: 0.8345
Epoch 8/10
3/3 [=================

2024-03-16 15:23:20.499783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:23:20.568347: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:23:20.791602: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 713ms/step
K-Fold Cross-Validation Results (5 folds):
Average Accuracy: 0.8561 ± 0.0100
Average Precision: 0.7445 ± 0.0094
Average Recall: 0.7509 ± 0.0232
Average F1-Score: 0.7443 ± 0.0133


In [78]:
# Train and evaluate based on train and test data
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_text.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_text, train_label, test_text, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics for text:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:28:55.763612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:28:55.952965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:28:56.191874: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:28:56.685013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:28:57.112980: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 3s 284ms/step - loss: 1.7350 - accuracy: 0.4908
Epoch 2/10
4/4 [==============================] - 1s 155ms/step - loss: 1.5545 - accuracy: 0.7670
Epoch 3/10
4/4 [==============================] - 0s 116ms/step - loss: 1.3612 - accuracy: 0.7997
Epoch 4/10
4/4 [==============================] - 0s 90ms/step - loss: 1.0152 - accuracy: 0.8158
Epoch 5/10
4/4 [==============================] - 0s 65ms/step - loss: 0.5962 - accuracy: 0.8242
Epoch 6/10
4/4 [==============================] - 0s 97ms/step - loss: 0.4972 - accuracy: 0.8352
Epoch 7/10
4/4 [==============================] - 0s 71ms/step - loss: 0.4454 - accuracy: 0.8505
Epoch 8/10
4/4 [==============================] - 0s 51ms/step - loss: 0.4104 - accuracy: 0.8609
Epoch 9/10
4/4 [==============================] - 0s 49ms/step - loss: 0.3779 - accuracy: 0.8721
Epoch 10/10
4/4 [==============================] - 0s 49ms/step - loss: 0.3460 - accuracy: 0.8858


2024-03-16 15:29:01.360368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:01.423680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:01.635429: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 687ms/step
Test Metrics for text:
Accuracy: 0.7674
Precision: 0.6824
Recall: 0.5699
F1 Score: 0.6011


In [79]:
# Train and evaluate based on train and test data
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_audio.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_audio, train_label, test_audio, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics for audio:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:29:19.002007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:19.206458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:19.534450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:20.015969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:20.462083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 4s 312ms/step - loss: 1.7700 - accuracy: 0.4690
Epoch 2/10
4/4 [==============================] - 1s 194ms/step - loss: 1.6635 - accuracy: 0.6039
Epoch 3/10
4/4 [==============================] - 0s 93ms/step - loss: 1.4158 - accuracy: 0.6021
Epoch 4/10
4/4 [==============================] - 0s 81ms/step - loss: 1.1188 - accuracy: 0.6089
Epoch 5/10
4/4 [==============================] - 0s 51ms/step - loss: 0.9897 - accuracy: 0.6407
Epoch 6/10
4/4 [==============================] - 0s 76ms/step - loss: 0.8986 - accuracy: 0.6642
Epoch 7/10
4/4 [==============================] - 0s 58ms/step - loss: 0.8717 - accuracy: 0.6721
Epoch 8/10
4/4 [==============================] - 0s 53ms/step - loss: 0.8383 - accuracy: 0.6793
Epoch 9/10
4/4 [==============================] - 0s 61ms/step - loss: 0.8123 - accuracy: 0.6870
Epoch 10/10
4/4 [==============================] - 0s 61ms/step - loss: 0.7883 - accuracy: 0.7036


2024-03-16 15:29:24.967784: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:25.034467: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:29:25.300778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 798ms/step
Test Metrics for audio:
Accuracy: 0.6666
Precision: 0.5647
Recall: 0.4334
F1 Score: 0.3776


In [81]:
# Train and evaluate based on train and test data
# Instantiate the model
early_fusion_model = EarlyFusionModel(input_shape=train_visual.shape[1:], num_classes=train_label.shape[2])

# Compile the model
early_fusion_model.compile(lr=0.001)

# Train and evaluate on the test dataset
history, test_metrics = early_fusion_model.train_and_evaluate_on_test(train_visual, train_label, test_visual, test_label, epochs=10, batch_size=32)

# Print the test metrics
print("Test Metrics for video:")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"F1 Score: {test_metrics['f1_score']:.4f}")


Epoch 1/10


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2024-03-16 15:30:02.716937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:30:02.920025: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:30:03.198556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:30:03.702355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:30:04.132755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 4s 320ms/step - loss: 1.7769 - accuracy: 0.4806
Epoch 2/10
4/4 [==============================] - 0s 110ms/step - loss: 1.7004 - accuracy: 0.5983
Epoch 3/10
4/4 [==============================] - 0s 110ms/step - loss: 1.4602 - accuracy: 0.5980
Epoch 4/10
4/4 [==============================] - 0s 109ms/step - loss: 1.1911 - accuracy: 0.5982
Epoch 5/10
4/4 [==============================] - 0s 79ms/step - loss: 1.1204 - accuracy: 0.6134
Epoch 6/10
4/4 [==============================] - 0s 56ms/step - loss: 0.9727 - accuracy: 0.6327
Epoch 7/10
4/4 [==============================] - 0s 49ms/step - loss: 0.9072 - accuracy: 0.6489
Epoch 8/10
4/4 [==============================] - 0s 53ms/step - loss: 0.9031 - accuracy: 0.6439
Epoch 9/10
4/4 [==============================] - 0s 48ms/step - loss: 0.8805 - accuracy: 0.6469
Epoch 10/10
4/4 [==============================] - 0s 84ms/step - loss: 0.8623 - accuracy: 0.6527


2024-03-16 15:30:08.522543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:30:08.591542: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-16 15:30:08.801152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 690ms/step
Test Metrics for video:
Accuracy: 0.6381
Precision: 0.1871
Recall: 0.3037
F1 Score: 0.2178


/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
